In [1]:
import pandas as pd 
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module=r"sklearn\.compose\._target")

import numpy as np

from pathlib import Path
from time import perf_counter

from sklearn.model_selection import TimeSeriesSplit
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# -----------------------------
# Config
# -----------------------------
DATA_PATH = "Preprocessed_Solar_Power_Data_v3.csv"
cons_col = "TOTALDEMAND"
irr_col  = "ALLSKY_SFC_SW_DWN"

irr_lag_max = 24     # irradiance lags
dem_lag_max = 24     # demand AR lags

n_splits_cv   = 3
random_state  = 42

#hyperparameters
K_LIST = [10, 15, 20, 25, 30, 40, 50, 60]
WEIGHTS_LIST = ["uniform", "distance"]

t0 = perf_counter()
df = pd.read_csv(DATA_PATH)

#Lag features
for L in range(1, irr_lag_max + 1):
    df[f"irr_lag_{L}"] = df[irr_col].shift(L)
for L in range(1, dem_lag_max + 1):
    df[f"dem_lag_{L}"] = df[cons_col].shift(L)

df = df.dropna().reset_index(drop=True)

#features
exog_cols = [
    "T2M", "T2MDEW", "T2MWET", "RH2M", "PS", "WS2M",
    "Basel Wind Gust", "Basel Wind Direction [10 m]",
    "Basel Precipitation Total", "Basel Wind Speed [10 m]", "Basel Cloud Cover Total",
    "HOUR_SIN", "HOUR_COS", "WEEKDAY_SIN", "WEEKDAY_COS",
]
irr_lags = [f"irr_lag_{i}" for i in range(1, irr_lag_max + 1)]
dem_lags = [f"dem_lag_{i}" for i in range(1, dem_lag_max + 1)]

feat_cols = [c for c in exog_cols + irr_lags + dem_lags if c in df.columns]
X = df[feat_cols].astype("float32")
y = df[cons_col].astype("float32")
print(f"Feature matrix shape: {X.shape}  | Target length: {len(y)}")

#Fixed settings for reduced runtime
k_select = min(50, max(25, X.shape[1] // 2))
pca_components = min(20, k_select)            

#Pipeline
def make_pipe(n_neighbors:int, weights:str) -> Pipeline:
    return Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", RobustScaler()),
        ("select", SelectKBest(score_func=mutual_info_regression, k=k_select)),
        ("pca", PCA(n_components=pca_components, svd_solver="auto", random_state=random_state)),
        ("knn", KNeighborsRegressor(n_neighbors=n_neighbors, weights=weights, algorithm="auto",  metric="minkowski", p=2)),
    ])

#TSCV grid search 
tscv = TimeSeriesSplit(n_splits=n_splits_cv)

def cv_mse(pipe: Pipeline) -> float:
    """Mean MSE across TSCV folds; returns inf if a fold errors."""
    mses = []
    for tr, te in tscv.split(X):
        try:
            pipe.fit(X.iloc[tr], y.iloc[tr].values)
            pred = pipe.predict(X.iloc[te])
            mses.append(mean_squared_error(y.iloc[te].values, pred))
        except Exception as e:
            return float("inf")
    return float(np.mean(mses))

best_cfg = None
best_mse = np.inf

for w in WEIGHTS_LIST:
    for k in K_LIST:
        pipe = make_pipe(n_neighbors=k, weights=w)
        mse = cv_mse(pipe)
        print(f"  k={k:2d}, weights={w:<8s} -> CV MSE={mse:,.1f}")
        if mse < best_mse:
            best_mse = mse
            best_cfg = (k, w)

#if best_cfg is None or not np.isfinite(best_mse):
#   raise RuntimeError("Search failed to find a valid configuration.")

k_best, w_best = best_cfg
print(f"\nBest config: k={k_best}, weights={w_best}  (CV MSE={best_mse:,.1f})")

best_model = make_pipe(n_neighbors=k_best, weights=w_best)

print("\nBest model pipeline structure:")
print(best_model)

# Access the actual KNN regressor inside the pipeline
knn_params = best_model.named_steps["knn"].get_params()
print("\nBest KNeighborsRegressor parameters:")
for key, val in knn_params.items():
    print(f"  {key}: {val}")

def eval_regression(y_true, y_pred, label=""):
    mse = mean_squared_error(y_true, y_pred)
    rmse = float(np.sqrt(mse))
    mae = mean_absolute_error(y_true, y_pred)
    r2  = r2_score(y_true, y_pred)
    print(f"[{label}] MSE={mse:.2f}  RMSE={rmse:.2f}  MAE={mae:.2f}  R²={r2:.3f}")
    return {"MSE": mse, "RMSE": rmse, "MAE": mae, "R2": r2}

print("\nFold-by-fold performance of best model:")
for i, (tr, te) in enumerate(tscv.split(X), 1):
    best_model.fit(X.iloc[tr], y.iloc[tr].values)
    pred = best_model.predict(X.iloc[te])
    _ = eval_regression(y.iloc[te].values, pred, f"Fold {i}")



Feature matrix shape: (48072, 63)  | Target length: 48072
  k=10, weights=uniform  -> CV MSE=109,589.0
  k=15, weights=uniform  -> CV MSE=113,117.4
  k=20, weights=uniform  -> CV MSE=116,705.3
  k=25, weights=uniform  -> CV MSE=120,075.5
  k=30, weights=uniform  -> CV MSE=123,123.9
  k=40, weights=uniform  -> CV MSE=129,008.1
  k=50, weights=uniform  -> CV MSE=134,004.3
  k=60, weights=uniform  -> CV MSE=138,292.8
  k=10, weights=distance -> CV MSE=107,083.9
  k=15, weights=distance -> CV MSE=110,419.7
  k=20, weights=distance -> CV MSE=113,448.1
  k=25, weights=distance -> CV MSE=116,518.7
  k=30, weights=distance -> CV MSE=119,324.8
  k=40, weights=distance -> CV MSE=124,691.3
  k=50, weights=distance -> CV MSE=129,290.3
  k=60, weights=distance -> CV MSE=133,247.7

Best config: k=10, weights=distance  (CV MSE=107,083.9)

Best model pipeline structure:
Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('scaler', RobustScaler()),
                ('select',